In [ ]:
import sys
sys.path.insert(0, '../')

import utils, evaluation
#from evaluation import load_gan
from data import fmap, Dataset
from JR_Scripts import dict_reader, time_toy_generator
from JR_Scripts import notebook_helper as nh
from model import WGanModel, WNGanModel, TemporalGanModelv3
from gan import TimeCosmoGAN
import metrics
import tensorflow as tf
import numpy as np
from PIL import Image
import imageio
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import importlib as imp

In [ ]:
redshift = ['z = 0.000', 'z = 0.111', 'z = 0.250', 'z = 0.428', 'z = 0.666', 'z = 1.000', 'z = 1.500', 'z = 2.333', 'z = 4.000', 'z = 9.000']

In [ ]:
params, model_folder = nh.load_params("models_64x64_C4/TWGANce:62d3_selu_sn10-10_0246r_Hlr3e5_bs4_v2ad_c+sf62_checkpoints/", cscs=True)
specific_model = 164800

In [ ]:
nh.print_param_dict(params)

In [ ]:
dset = nh.get_dataset(500, 512, params, shuffle=True, cscs=True)

In [ ]:
gan = evaluation.load_gan(model_folder, GANtype=TimeCosmoGAN)

In [ ]:
main_classes, contained_classes = nh.get_main_and_contained_classes(params)

In [ ]:
def visual_comparison_fake_real_compact(dset, gan, chpt):
    series = dset.get_samples(100)[np.random.randint(100)]
    series = np.transpose(series, [2, 0, 1])
    img_series = np.array(nh.gen_images_10_time_steps(gan, chpt))
    main_classes, contained_classes = nh.get_main_and_contained_classes(gan.params)
    marker = nh.gen_contained_marker(img_series, series, contained_classes)
    img_series = np.flip(img_series[main_classes], 0)
    series = np.flip(series[main_classes], 0)
    marker = np.flip(marker[main_classes], 0)
    fig, ax = plt.subplots(figsize=(128, 16))
    ax.imshow(np.vstack([np.hstack(img_series), np.hstack(marker), np.hstack(series)]), interpolation=None)
    # ax.imshow(np.hstack(series), interpolation=None)
    plt.tight_layout()
    return img_series, series

In [ ]:
fake_img, real_img = visual_comparison_fake_real_compact(dset, gan, specific_model)

In [ ]:
fake_img, real_img = visual_comparison_fake_real_compact(dset, gan, specific_model)

In [ ]:
fake_img, real_img = visual_comparison_fake_real_compact(dset, gan, specific_model)

In [ ]:
for i in range(10):
    fake_img, real_img = visual_comparison_fake_real_compact(dset, gan, specific_model)

In [ ]:
t = params['time']['classes']
images = nh.gen_fake_images(gan, 5000, t, checkpoint=specific_model)
images = params['cosmology']['backward_map'](images)

In [ ]:
plt.figure(figsize = (20,2))
plt.imshow(np.hstack(params['cosmology']['forward_map'](images[12:30])), interpolation=None)
#images.shape

In [ ]:
nh.get_scaling(gan.params['time']['classes'], gan.params['time']['class_weights'])

In [ ]:
data = dset.get_samples(5000)
data = nh.reshape_data_to_old_format(data, params)
data = params['cosmology']['backward_map'](data)
data.shape

# Single Timestep Data Analysis

## Mass Histograms

### Progression over time

In [ ]:
#imp.reload(nh)

In [ ]:
cmap = nh.get_cmap('viridis', params['time']['num_classes'])
# cmap = cmap[1:-1]

lim_mass = nh.get_lim_mass(images)
nh.plot_mass_hist(images, "Fake", lim_mass, params, cmap)
nh.plot_mass_hist(data, "Real", lim_mass, params, cmap)

### Real-Fake timewise comparisons

In [ ]:
lim_mass = nh.get_lim_mass(data)
nh.plot_real_vs_fake_mass_hists(data, images, lim_mass, params)

## Peak Histograms

### Progression over time

In [ ]:
_, _, lim = metrics.peak_count_hist(data)
lim = (lim[0], lim[1] + 1)
nh.peak_hist_over_time(data, images, params, lim=lim, title_a="Real Data", title_b="Fake Data")

### Real-Fake timewise comparison

In [ ]:
nh.peak_hist_a_vs_b(data, images, params, lim=lim, label_a="Real", label_b="Fake")

## Power Spectral Densities

### Progression over time

In [ ]:
nh.power_spectral_densities(data, params, data_name="Real Data")
nh.power_spectral_densities(images, params, data_name="Fake Data")

In [ ]:
#imp.reload(nh)

### Real-Fake timewise comparison

In [ ]:
nh.power_spectral_density_a_vs_b(data, images, params)